## 爬留言

In [1]:
import time
import numpy as np
import pandas as pd
from utils import mysqlDatabase, dcardApi

In [2]:
class getComments:
    def __init__(self, database_username, database_password, database_ip, database_name, base_url, popular, max_limit):
        self.database_username = database_username
        self.database_password = database_password
        self.database_ip = database_ip
        self.database_name = database_name
        self.base_url = base_url
        self.popular = popular
        self.max_limit = max_limit
    def get_postid_list(self, MysqlDatabase, sql):
        df = MysqlDatabase.select_table(sql)
        postid_list = df['postid'].tolist()
        return postid_list
    def get_comment(self, DcardApi, postid):
        df = DcardApi.get_Dcard_posts_comments(postid)
        return df
    def main(self):
        '''
        實作
        '''
        MysqlDatabase = mysqlDatabase(self.database_username, self.database_password, self.database_ip, self.database_name)
        DcardApi = dcardApi(self.base_url, self.popular, self.max_limit)
        ## 文章列表
        sql = '''SELECT distinct postid, title FROM Bigdata.dcard_posts where commentCount >= 30'''
        postid_list = self.get_postid_list(MysqlDatabase, sql)
        ## 打api拿留言
        for postid in postid_list:
            # print(postid)
            df_comment = self.get_comment(DcardApi, postid)
            break
        df_comment = df_comment[['id', 'anonymous', 'postId', 'createdAt', 'floor', 'content', 'likeCount', 'withNickname', 'gender', 'school', 'department']]
        MysqlDatabase.insert_table(df_comment, table_name='dcard_comments')
        return df_comment

In [8]:
if __name__ == '__main__':
    database_username = 'jackyfu'
    database_password = 'data8756'
    database_ip       = 'sg2nlmysql29plsk.secureserver.net'
    database_name     = 'Bigdata'
    base_url = 'https://www.dcard.tw/service/api/v2'
    popular = 'false'
    max_limit = '100'
    ###
    GetComments = getComments(database_username, database_password, database_ip, database_name, base_url, popular, max_limit)
    df_comment = GetComments.main()

Successfully select from Bigdata table
Successfully insert into Bigdata table: dcard_comments


In [9]:
df_comment.head()

,id,anonymous,postId,createdAt,floor,content,likeCount,withNickname,gender,school,department
0,afec17bd-b835-46f8-a5db-437b021a1c27,True,236808962,2021-08-25T17:28:25.019Z,1,欸不是要賣奶不要賣一半‎꜀( ꜆˙꒳​˙)꜆,21,False,M,長庚大學,NaN
1,cab1968a-c5db-40b3-be64-eff72e9d1e09,False,236808962,2021-08-25T17:28:48.662Z,2,騷擾你也是我的自由 ...,10,True,M,亞洲董神,larry
2,2f20fc10-ca1b-4dca-9bef-dee0ad86054d,False,236808962,2021-08-25T17:28:55.721Z,3,欸不是要賣奶不要賣一半‎꜀( ꜆˙꒳​˙)꜆,14,True,M,拿鐵大學 不想上班所,montag
3,e2bec33b-c995-4941-a13b-2b8f68264de6,False,236808962,2021-08-25T17:29:02.788Z,4,欸不是要賣奶不要賣一半‎꜀( ꜆˙꒳​˙)꜆,8,True,M,亞洲董神,larry
4,93cbb45b-0be8-4809-b953-c987b376261b,False,236808962,2021-08-25T17:30:15.348Z,5,欸不是要賣奶不要賣一半‎꜀( ꜆˙꒳​˙)꜆,9,True,M,Eligos,devil_egg
